<a href="https://colab.research.google.com/github/entenam/TF-Intro/blob/main/CatsVDogsTrLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

**IMPORTANT NOTE:** This notebook is designed to run as a Colab. Click the button on top that says, `Open in Colab`, to run this notebook as a Colab. Running the notebook on your local machine might result in some of the code blocks throwing errors.

In [2]:
import os

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model

!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
  


--2022-01-19 06:12:54--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.135.128, 74.125.142.128, 74.125.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.135.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   135MB/s    in 0.6s    

2022-01-19 06:12:55 (135 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [3]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
  layer.trainable = False
  
# pre_trained_model.summary()

last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 7, 7, 768)


In [4]:
from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense  (1, activation='sigmoid')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(optimizer = RMSprop(learning_rate=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import zipfile


# Define our example directories and files
base_dir = '/content/drive/MyDrive/AI Basics with Tensorflow/CATS-v-DOGS/cats-v-dogs'

train_dir = os.path.join( base_dir, 'training')
validation_dir = os.path.join( base_dir, 'testing')


train_cats_dir = os.path.join(train_dir, 'cats') # Directory with our training cat pictures
train_dogs_dir = os.path.join(train_dir, 'dogs') # Directory with our training dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats') # Directory with our validation cat pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')# Directory with our validation dog pictures

train_cat_fnames = os.listdir(train_cats_dir)
train_dog_fnames = os.listdir(train_dogs_dir)

print(train_cats_dir)

/content/drive/MyDrive/AI Basics with Tensorflow/CATS-v-DOGS/cats-v-dogs/training/cats


In [6]:
# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 20,
                                                    class_mode = 'binary', 
                                                    target_size = (150, 150))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 20,
                                                          class_mode  = 'binary', 
                                                          target_size = (150, 150))

Found 22475 images belonging to 2 classes.
Found 2523 images belonging to 2 classes.


In [7]:
#Save Model Checkpoint to Google Drive During Training
checkpoint_path = "/content/drive/My Drive/AI Basics with Tensorflow/Cats-v-DogsTrLearning/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [12]:
history = model.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 100,
            epochs = 20,
            validation_steps = 50,
            verbose = 2,
            callbacks = [cp_callback])

Epoch 1/20

Epoch 00001: saving model to /content/drive/My Drive/AI Basics with Tensorflow/Cats-v-DogsTrLearning/cp.ckpt
100/100 - 925s - loss: 0.3682 - accuracy: 0.8520 - val_loss: 0.1203 - val_accuracy: 0.9470 - 925s/epoch - 9s/step
Epoch 2/20

Epoch 00002: saving model to /content/drive/My Drive/AI Basics with Tensorflow/Cats-v-DogsTrLearning/cp.ckpt
100/100 - 745s - loss: 0.2299 - accuracy: 0.9145 - val_loss: 0.1100 - val_accuracy: 0.9550 - 745s/epoch - 7s/step
Epoch 3/20

Epoch 00003: saving model to /content/drive/My Drive/AI Basics with Tensorflow/Cats-v-DogsTrLearning/cp.ckpt
100/100 - 628s - loss: 0.2248 - accuracy: 0.9135 - val_loss: 0.0740 - val_accuracy: 0.9680 - 628s/epoch - 6s/step
Epoch 4/20


/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only


Epoch 00004: saving model to /content/drive/My Drive/AI Basics with Tensorflow/Cats-v-DogsTrLearning/cp.ckpt
100/100 - 547s - loss: 0.2195 - accuracy: 0.9180 - val_loss: 0.0963 - val_accuracy: 0.9660 - 547s/epoch - 5s/step
Epoch 5/20

Epoch 00005: saving model to /content/drive/My Drive/AI Basics with Tensorflow/Cats-v-DogsTrLearning/cp.ckpt
100/100 - 496s - loss: 0.2004 - accuracy: 0.9313 - val_loss: 0.1242 - val_accuracy: 0.9560 - 496s/epoch - 5s/step
Epoch 6/20

Epoch 00006: saving model to /content/drive/My Drive/AI Basics with Tensorflow/Cats-v-DogsTrLearning/cp.ckpt
100/100 - 445s - loss: 0.1832 - accuracy: 0.9315 - val_loss: 0.2130 - val_accuracy: 0.9400 - 445s/epoch - 4s/step
Epoch 7/20

Epoch 00007: saving model to /content/drive/My Drive/AI Basics with Tensorflow/Cats-v-DogsTrLearning/cp.ckpt
100/100 - 390s - loss: 0.2205 - accuracy: 0.9223 - val_loss: 0.0688 - val_accuracy: 0.9690 - 390s/epoch - 4s/step
Epoch 8/20

Epoch 00008: saving model to /content/drive/My Drive/AI Bas

In [ ]:
model.load_weights(checkpoint_path)
model.evaluate(validation_generator)

In [24]:
img = image.load_img('/content/drive/MyDrive/AI Basics with Tensorflow/CATS-v-DOGS/cats-v-dogs/training/cats/0.jpg', target_size= (150,150,3))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
model.predict(x)

array([[0.]], dtype=float32)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

In [32]:
# Here's a codeblock just for fun. You should be able to upload an image here 
# and have it classified without crashing

import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(150,150,3))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  #images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a dog")
  else:
    print(fn + " is a cat")

Saving 10008dog.jpg to 10008dog.jpg
Saving 10003dog.jpg to 10003dog.jpg
[0.]
10008dog.jpg is a cat
[0.]
10003dog.jpg is a cat
